In [ ]:
!pip install pystan convertdate lunarcalendar holidays pyarrow google-cloud-bigquery google-cloud-bigquery-storage pandas_gbq pandas prophet==1.0.1 

In [ ]:
from prophet import Prophet
import numpy as np
import pandas as pd
import pandas_gbq
from datetime import date
from google.cloud import bigquery
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [ ]:
client = bigquery.Client(project="timeseries-batch-forecasting")
sql = """
SELECT
  date as ds,
  item_description,
  SUM(bottles_sold) as y
FROM
  `iowa_liquor_retail_sales.sales` # TABLESAMPLE SYSTEM (10 PERCENT)
WHERE date > DATE("2021-12-31") AND item_description LIKE 'J%'
GROUP BY
  date,
  item_description
ORDER BY
  item_description,
  date;
"""
query_job = client.query(sql).to_dataframe()

In [ ]:
sales_data = query_job.astype({'y': 'int32'})
grouped = sales_data.groupby(sales_data.item_description)
grouped.head()

In [ ]:
def run_prophet(timeseries):
    if len(timeseries[1]) < 2:
        return
    model = Prophet(daily_seasonality=False, yearly_seasonality=False, weekly_seasonality=False)
    model.fit(timeseries[1])
    forecast_def = model.make_future_dataframe(periods=30, include_history=False)
    forecast = model.predict(forecast_def)
    return (timeseries[0], forecast)

In [ ]:
p = Pool(cpu_count())
predictions = list(tqdm(p.imap(run_prophet, grouped), total=grouped.ngroups))
p.close()
p.join()

In [ ]:
predictions[0]

In [ ]:
predictions_clean = []
for p in predictions:
    if p is None:
        continue
    p[1]['Product'] = p[0]
    predictions_clean.append(p[1])
pandas_gbq.to_gbq(dataframe=pd.concat(predictions_clean), destination_table="timeseries-batch-forecasting.forecast_results.from_notebook_{}".format(date.today()), if_exists='replace', project_id="timeseries-batch-forecasting")

In [2]:
call = [1,2,3,4]
result = [x+1 for x in call]
result

[2, 3, 4, 5]